In [1]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.7 MB/s eta 0:00:00


In [2]:
!pip install openpyxl pandas

In [ ]:
import pandas as pd

# Excelファイル全体の読み込み
# df = pd.read_excel('preExpData.xlsx', sheet_name='特徴表')
df = pd.read_excel('preExpData.xlsx')

# データの確認
print(df)

                 列1  列2 列22  列3 列4   列5   列6   列7
0           観光スポット名  自然  風景  文化  食  買い物  利用費   ID
1           []旧梅津倉庫   0   0   0  0    0    0  -24
2         []旧ホテル中央荘   0   0   0  0    0    0  -23
3   []日本最古のコンクリート電柱   0   0   0  0    0    0  -22
4          []美容室あみん   0   0   0  0    0    0  -21
..              ...  ..  ..  .. ..  ...  ...  ...
81              八幡坂   3   4   1  1    1    0   56
82             船魂神社   3   2   2  1    1    0   57
83             元町公園   3   3   2  1    1    0   58
84              弥生坂   3   2   1  1    1    0   59
85         函館市熱帯植物園   5   3   1  1    1  300   60

[86 rows x 8 columns]


In [ ]:
import math
import numpy as np
import random
import xlrd

In [ ]:
from deap import base
from deap import creator
from deap import tools

In [ ]:
from google.colab import files

# ファイルをアップロードする
uploaded = files.upload()

# アップロードされたファイルを確認
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving preExpData.xlsx to preExpData.xlsx
User uploaded file "preExpData.xlsx" with length 51170 bytes


In [ ]:
# スポットデータを格納するための
# class spotData:
#     nature = 0
#     landscape = 0
#     culture = 0
#     food = 0
#     shopping = 0
#     admission = 0


# PROBLEM = "RouteGeneration"
# NOBJ = 3
# K = 10 # 目的関数の数？不要？
# NDIM = NOBJ + K - 1
# P = 12
# H = math.factorial(NOBJ + P - 1)/ (math.factorial(P) * math.factorial(NOBJ - 1))
#
# MU = int(H + (4 - H % 4)) # 個体数
# NGEN = 100                # 何世代まで更新するか
# CXPB = 0.9                # 交叉率
# MUTPB = 0.1               # 変異率
#
# ref_points = tools.uniform_reference_points(NOBJ, P)

allSpotNum = 61 # 全ての観光スポット数
MU = 5 # 個体数（観光ルート数）

# 観光スポットの情報を格納する箱を用意
dtype = [("id","u1"),("nature", "u1"), ("landscape","u1"),("culture","u1"),("food","u1"),("shopping","u1"),("admission","u2")]
spotData = np.zeros(61, dtype = dtype)

itype = [("courseId","U12"),("nature", "f2"), ("landscape","f2"),("culture","f2"),("food","f2"),("shopping","f2"),("admission","u2")]
individuals = np.zeros(MU, dtype = itype)

# Excelファイル全体の読み込み
wb = pd.read_excel('preExpData.xlsx')
# Excelファイル内の特定のシートの読み込み
featureSheet = pd.read_excel('preExpData.xlsx', sheet_name='特徴表')

# 各特徴のデータ列を取得
idCol = featureSheet.iloc[25:86, 7].astype(str).values  # 'id' 列の値を取得し、文字列に変換
natureCol = featureSheet.iloc[25:86, 1].astype(str).values  # 'nature' 列の値を取得し、文字列に変換
landscapeCol = featureSheet.iloc[25:86, 2].astype(str).values  # 'landscape' 列の値を取得し、文字列に変換
cultureCol = featureSheet.iloc[25:86, 3].astype(str).values  # 'culture' 列の値を取得し、文字列に変換
foodCol = featureSheet.iloc[25:86, 4].astype(str).values  # 'food' 列の値を取得し、文字列に変換
shoppingCol = featureSheet.iloc[25:86, 5].astype(str).values  # 'shopping' 列の値を取得し、文字列に変換
admissionCol = featureSheet.iloc[25:86, 6].astype(str).values  # 'admission' 列の値を取得し、文字列に変換

print("idCol")
print(idCol)

# 各データ列を観光スポットの情報を保存する箱に格納
spotData["id"] = idCol
spotData["nature"] = natureCol
spotData["landscape"] = landscapeCol
spotData["culture"] = cultureCol
spotData["food"] = foodCol
spotData["shopping"] = shoppingCol
spotData["admission"] = admissionCol

print(len(spotData))
# print(spotData[0]["id"])


# 初期解を生成する

# 一定範囲からランダムに観光スポット数を決定する
# 今回は省略

print("('courseNum', 自然, 風景, 文化, 食, 買い物, 利用費)")

# 上記で作成した範囲からランダムな値を選択する
for i in range(MU):
    spotNum = random.randint(4, 8)
    print(spotNum)

    # 全スポットから重複なしのランダムにスポットを選択する
    # range(x) は 0 から x-1 までの値を指す
    route = random.sample(range(allSpotNum), k=spotNum)
    print(route)

    individuals[i]["courseId"] = "course" + str(i)

    for j in range(spotNum):
        # コース内の各特徴を評価する(現状はただの足し算)
        individuals[i]["nature"] += spotData[route[j]]["nature"] - 1.5
        individuals[i]["landscape"] += spotData[route[j]]["landscape"] - 1.5
        individuals[i]["culture"] += spotData[route[j]]["culture"] - 1.5
        individuals[i]["food"] += spotData[route[j]]["food"] - 1.5
        individuals[i]["shopping"] += spotData[route[j]]["shopping"] - 1.5
        individuals[i]["admission"] += spotData[route[j]]["admission"]

    print(individuals[i])


# 評価？

ref_points = tools.uniform_reference_points(4, 4)
# print(ref_points)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# print("H : " + str(H))
print("Success")


idCol
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60']
61
('courseNum', 自然, 風景, 文化, 食, 買い物, 利用費)
8
[2, 28, 39, 18, 6, 55, 22, 43]
('course0', 1., 7., 7., -3., -4., 400)
8
[10, 22, 23, 18, 31, 39, 48, 43]
('course1', 1., 5., 5., -1., -2., 300)
5
[45, 46, 34, 36, 5]
('course2', 1.5, 1.5, 4.5, -0.5, 2.5, 300)
6
[49, 50, 37, 42, 12, 48]
('course3', 5., 10., 0., 2., 1., 1800)
5
[49, 11, 7, 41, 22]
('course4', 0.5, 2.5, 3.5, 0.5, -2.5, 300)
Success


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
